TensorFlow es una plataforma open source para ML, contiene todo un conjunto de herramientas que permiten a los desarrolladores utilizar diferentes técnicas de aprendizaje máquina y desplegarlas en entornos de desarrollo.

In [48]:
from __future__ import print_function
import tensorflow as tf
import numpy as np

In [49]:
# En tensorflow usamos sesiones y vamos agregando elementos
a = tf.constant(10)
b = tf.constant(20)

with tf.Session() as sess:
    print("a: %i" % sess.run(a), "b: %i" % sess.run(b))
    print("Addition with constants: %i" % sess.run(a+b))
    print("Multiplication with constants: %i" % sess.run(a*b))

a: 10 b: 20
Addition with constants: 30
Multiplication with constants: 200


La base de MNIST contiene catacteres escritos. Contiene 60,000 ejemplos para entrenamiento y 10,000 ejemplos para probar. Los dígitos fueron normalizados y centrados en imágenes del mismo tamaño (28x28 pixeles) con valores de 0 y 1 indicando blanco o negro cada pixel
![alt text](mnist.png)

In [50]:
# Importamos datos de la base MNIST
from tensorflow.keras.datasets import mnist 
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print('Tamaño de la base de entrenamiento: ', X_train.shape) # 60,000 imagenes 28 x 28 pixeles
print('Tamaño de la base de prueba: ', X_test.shape) # 10,000 imagenes de 28x28 pixeles

# Convertir a un vector de 1 x 784 pixeles
image_vector_size = 28*28
X_train = X_train.reshape(X_train.shape[0], image_vector_size)
X_test = X_test.reshape(X_test.shape[0], image_vector_size)

print(X_train[:5])

print('Etiquetas sin procesar: ', y_train[:5])

# Convertir las etiquetas a categorías
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print('Categorías procesadas: ', y_train[:5])

Tamaño de la base de entrenamiento:  (60000, 28, 28)
Tamaño de la base de prueba:  (10000, 28, 28)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Etiquetas sin procesar:  [5 0 4 1 9]
Categorías procesadas:  [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


Ahora procedemos a construir el modelo, con la cantidad de capas y neuronas que diseñamos

In [43]:
# Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [44]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [45]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [46]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    for step in range(1, num_steps + 1):
        arr = np.arange(X_train.shape[0])
        np.random.shuffle(arr)
        for index in range(0, X_train.shape[0], batch_size):
            batch_x = X_train[arr[index:index+batch_size]]
            batch_y = y_train[arr[index:index+batch_size]]
            # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            if step % display_step == 0 or step == 1:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                     Y: batch_y})
                print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test,
                                      Y: y_test}))

Step 1, Minibatch Loss= 2321288.7500, Training Accuracy= 0.422
Step 1, Minibatch Loss= 2744206.7500, Training Accuracy= 0.492
Step 1, Minibatch Loss= 3321511.5000, Training Accuracy= 0.273
Step 1, Minibatch Loss= 5076732.5000, Training Accuracy= 0.227
Step 1, Minibatch Loss= 3051876.7500, Training Accuracy= 0.383
Step 1, Minibatch Loss= 1464942.6250, Training Accuracy= 0.562
Step 1, Minibatch Loss= 1385307.0000, Training Accuracy= 0.570
Step 1, Minibatch Loss= 1815275.2500, Training Accuracy= 0.422
Step 1, Minibatch Loss= 1234498.3750, Training Accuracy= 0.586
Step 1, Minibatch Loss= 823001.7500, Training Accuracy= 0.664
Step 1, Minibatch Loss= 861072.6875, Training Accuracy= 0.680
Step 1, Minibatch Loss= 1038990.8125, Training Accuracy= 0.656
Step 1, Minibatch Loss= 916530.3750, Training Accuracy= 0.719
Step 1, Minibatch Loss= 765652.3750, Training Accuracy= 0.680
Step 1, Minibatch Loss= 804554.5625, Training Accuracy= 0.742
Step 1, Minibatch Loss= 648358.8125, Training Accuracy= 0.70

Step 1, Minibatch Loss= 70201.6016, Training Accuracy= 0.875
Step 1, Minibatch Loss= 74183.5469, Training Accuracy= 0.797
Step 1, Minibatch Loss= 47484.6836, Training Accuracy= 0.852
Step 1, Minibatch Loss= 50236.3594, Training Accuracy= 0.859
Step 1, Minibatch Loss= 43023.5859, Training Accuracy= 0.875
Step 1, Minibatch Loss= 35129.7852, Training Accuracy= 0.883
Step 1, Minibatch Loss= 28812.1016, Training Accuracy= 0.922
Step 1, Minibatch Loss= 33069.6211, Training Accuracy= 0.906
Step 1, Minibatch Loss= 58502.2773, Training Accuracy= 0.891
Step 1, Minibatch Loss= 41682.7148, Training Accuracy= 0.852
Step 1, Minibatch Loss= 43963.5938, Training Accuracy= 0.844
Step 1, Minibatch Loss= 47369.7031, Training Accuracy= 0.898
Step 1, Minibatch Loss= 46847.8086, Training Accuracy= 0.859
Step 1, Minibatch Loss= 37270.9805, Training Accuracy= 0.875
Step 1, Minibatch Loss= 28224.0527, Training Accuracy= 0.883
Step 1, Minibatch Loss= 45072.7695, Training Accuracy= 0.875
Step 1, Minibatch Loss= 

Step 1, Minibatch Loss= 34153.9844, Training Accuracy= 0.852
Step 1, Minibatch Loss= 20125.0195, Training Accuracy= 0.828
Step 1, Minibatch Loss= 11265.1270, Training Accuracy= 0.930
Step 1, Minibatch Loss= 9749.3193, Training Accuracy= 0.891
Step 1, Minibatch Loss= 20130.3984, Training Accuracy= 0.883
Step 1, Minibatch Loss= 21376.6719, Training Accuracy= 0.891
Step 1, Minibatch Loss= 34262.3086, Training Accuracy= 0.836
Step 1, Minibatch Loss= 10151.2109, Training Accuracy= 0.891
Step 1, Minibatch Loss= 5486.3516, Training Accuracy= 0.906
Step 1, Minibatch Loss= 24530.6211, Training Accuracy= 0.867
Step 1, Minibatch Loss= 39812.3203, Training Accuracy= 0.859
Step 1, Minibatch Loss= 12191.5508, Training Accuracy= 0.875
Step 1, Minibatch Loss= 12579.2891, Training Accuracy= 0.906
Step 1, Minibatch Loss= 17177.6328, Training Accuracy= 0.859
Step 1, Minibatch Loss= 22301.2930, Training Accuracy= 0.883
Step 1, Minibatch Loss= 16597.6250, Training Accuracy= 0.906
Step 1, Minibatch Loss= 28

Step 100, Minibatch Loss= 22918.8770, Training Accuracy= 0.883
Step 100, Minibatch Loss= 14903.9941, Training Accuracy= 0.898
Step 100, Minibatch Loss= 9471.1104, Training Accuracy= 0.930
Step 100, Minibatch Loss= 13478.7168, Training Accuracy= 0.875
Step 100, Minibatch Loss= 15388.6465, Training Accuracy= 0.883
Step 100, Minibatch Loss= 14042.4727, Training Accuracy= 0.938
Step 100, Minibatch Loss= 20024.7246, Training Accuracy= 0.922
Step 100, Minibatch Loss= 20941.6719, Training Accuracy= 0.844
Step 100, Minibatch Loss= 11067.1807, Training Accuracy= 0.898
Step 100, Minibatch Loss= 9830.9844, Training Accuracy= 0.938
Step 100, Minibatch Loss= 7946.7261, Training Accuracy= 0.922
Step 100, Minibatch Loss= 11759.9258, Training Accuracy= 0.867
Step 100, Minibatch Loss= 16440.3828, Training Accuracy= 0.898
Step 100, Minibatch Loss= 8892.1348, Training Accuracy= 0.898
Step 100, Minibatch Loss= 19065.4590, Training Accuracy= 0.906
Step 100, Minibatch Loss= 29066.2773, Training Accuracy= 0.

Step 100, Minibatch Loss= 6298.0693, Training Accuracy= 0.922
Step 100, Minibatch Loss= 15788.4150, Training Accuracy= 0.906
Step 100, Minibatch Loss= 7146.8145, Training Accuracy= 0.922
Step 100, Minibatch Loss= 11251.4219, Training Accuracy= 0.906
Step 100, Minibatch Loss= 11380.2139, Training Accuracy= 0.914
Step 100, Minibatch Loss= 8840.7607, Training Accuracy= 0.922
Step 100, Minibatch Loss= 17691.1875, Training Accuracy= 0.906
Step 100, Minibatch Loss= 16783.8906, Training Accuracy= 0.914
Step 100, Minibatch Loss= 21232.5605, Training Accuracy= 0.906
Step 100, Minibatch Loss= 16321.0908, Training Accuracy= 0.898
Step 100, Minibatch Loss= 19445.2539, Training Accuracy= 0.875
Step 100, Minibatch Loss= 15366.6240, Training Accuracy= 0.891
Step 100, Minibatch Loss= 5250.7744, Training Accuracy= 0.953
Step 100, Minibatch Loss= 18906.4277, Training Accuracy= 0.875
Step 100, Minibatch Loss= 9767.6523, Training Accuracy= 0.906
Step 100, Minibatch Loss= 14976.6826, Training Accuracy= 0.8

...

Step 200, Minibatch Loss= 15259.6572, Training Accuracy= 0.883
Step 200, Minibatch Loss= 10617.7559, Training Accuracy= 0.914
Step 200, Minibatch Loss= 38605.8594, Training Accuracy= 0.883
Step 200, Minibatch Loss= 12530.7480, Training Accuracy= 0.938
Step 200, Minibatch Loss= 5569.3623, Training Accuracy= 0.930
Step 200, Minibatch Loss= 9129.3037, Training Accuracy= 0.898
Step 200, Minibatch Loss= 17006.6758, Training Accuracy= 0.922
Step 200, Minibatch Loss= 39253.0742, Training Accuracy= 0.852
Step 200, Minibatch Loss= 11803.7959, Training Accuracy= 0.891
Step 200, Minibatch Loss= 20985.8555, Training Accuracy= 0.852
Step 200, Minibatch Loss= 8484.7178, Training Accuracy= 0.875
Step 200, Minibatch Loss= 27587.0977, Training Accuracy= 0.867
Step 200, Minibatch Loss= 30250.0508, Training Accuracy= 0.875
Step 200, Minibatch Loss= 5270.2178, Training Accuracy= 0.938
Step 200, Minibatch Loss= 6191.5215, Training Accuracy= 0.898
Step 200, Minibatch Loss= 13429.2520, Training Accuracy= 0.9

Step 300, Minibatch Loss= 375773.8125, Training Accuracy= 0.914
Step 300, Minibatch Loss= 137953.1875, Training Accuracy= 0.930
Step 300, Minibatch Loss= 365812.6562, Training Accuracy= 0.914
Step 300, Minibatch Loss= 81880.8359, Training Accuracy= 0.953
Step 300, Minibatch Loss= 158940.4844, Training Accuracy= 0.914
Step 300, Minibatch Loss= 363222.9062, Training Accuracy= 0.906
Step 300, Minibatch Loss= 254253.1250, Training Accuracy= 0.922
Step 300, Minibatch Loss= 568155.7500, Training Accuracy= 0.875
Step 300, Minibatch Loss= 231705.6250, Training Accuracy= 0.953
Step 300, Minibatch Loss= 210813.4688, Training Accuracy= 0.898
Step 300, Minibatch Loss= 59500.4062, Training Accuracy= 0.945
Step 300, Minibatch Loss= 66316.3984, Training Accuracy= 0.953
Step 300, Minibatch Loss= 250027.7656, Training Accuracy= 0.898
Step 300, Minibatch Loss= 208714.3594, Training Accuracy= 0.906
Step 300, Minibatch Loss= 293197.6562, Training Accuracy= 0.906
Step 300, Minibatch Loss= 431460.3125, Trai

Step 300, Minibatch Loss= 228334.4688, Training Accuracy= 0.883
Step 300, Minibatch Loss= 162170.0312, Training Accuracy= 0.906
Step 300, Minibatch Loss= 229108.9219, Training Accuracy= 0.922
Step 300, Minibatch Loss= 90373.7422, Training Accuracy= 0.914
Step 300, Minibatch Loss= 145778.7812, Training Accuracy= 0.914
Step 300, Minibatch Loss= 22317.3105, Training Accuracy= 0.953
Step 300, Minibatch Loss= 188406.4844, Training Accuracy= 0.898
Step 300, Minibatch Loss= 140996.5625, Training Accuracy= 0.914
Step 300, Minibatch Loss= 39592.1328, Training Accuracy= 0.953
Step 300, Minibatch Loss= 309055.2188, Training Accuracy= 0.891
Step 300, Minibatch Loss= 206741.2500, Training Accuracy= 0.898
Step 300, Minibatch Loss= 173760.9688, Training Accuracy= 0.922
Step 300, Minibatch Loss= 206280.1562, Training Accuracy= 0.867
Step 300, Minibatch Loss= 46454.5586, Training Accuracy= 0.930
Step 300, Minibatch Loss= 337112.5625, Training Accuracy= 0.898
Step 300, Minibatch Loss= 234246.5156, Train

Step 300, Minibatch Loss= 116379.0312, Training Accuracy= 0.922
Step 300, Minibatch Loss= 170850.4062, Training Accuracy= 0.906
Step 300, Minibatch Loss= 84192.1875, Training Accuracy= 0.906
Step 300, Minibatch Loss= 28478.6719, Training Accuracy= 0.961
Step 300, Minibatch Loss= 221911.7188, Training Accuracy= 0.883
Step 300, Minibatch Loss= 101225.2812, Training Accuracy= 0.891
Step 300, Minibatch Loss= 91067.5469, Training Accuracy= 0.938
Step 300, Minibatch Loss= 186626.8125, Training Accuracy= 0.891
Step 300, Minibatch Loss= 32237.9980, Training Accuracy= 0.922
Step 300, Minibatch Loss= 74055.6875, Training Accuracy= 0.906
Step 300, Minibatch Loss= 177119.8750, Training Accuracy= 0.914
Step 300, Minibatch Loss= 140126.2812, Training Accuracy= 0.945
Step 300, Minibatch Loss= 59083.4414, Training Accuracy= 0.945
Step 300, Minibatch Loss= 102498.1797, Training Accuracy= 0.898
Step 300, Minibatch Loss= 144428.1094, Training Accuracy= 0.875
Step 300, Minibatch Loss= 72473.7969, Training

Step 300, Minibatch Loss= 31693.1387, Training Accuracy= 0.930
Step 300, Minibatch Loss= 87692.8594, Training Accuracy= 0.938
Step 300, Minibatch Loss= 78780.5469, Training Accuracy= 0.945
Step 300, Minibatch Loss= 75988.9766, Training Accuracy= 0.938
Step 300, Minibatch Loss= 34922.0195, Training Accuracy= 0.930
Step 300, Minibatch Loss= 78068.0000, Training Accuracy= 0.914
Step 300, Minibatch Loss= 103019.1172, Training Accuracy= 0.883
Step 300, Minibatch Loss= 125411.9922, Training Accuracy= 0.836
Step 300, Minibatch Loss= 38611.2461, Training Accuracy= 0.930
Step 300, Minibatch Loss= 26318.4980, Training Accuracy= 0.938
Step 300, Minibatch Loss= 147695.5781, Training Accuracy= 0.930
Step 300, Minibatch Loss= 95177.9219, Training Accuracy= 0.875
Step 300, Minibatch Loss= 45157.7344, Training Accuracy= 0.922
Step 300, Minibatch Loss= 26128.1973, Training Accuracy= 0.945
Step 300, Minibatch Loss= 77683.6875, Training Accuracy= 0.930
Step 300, Minibatch Loss= 40219.0469, Training Accur